In [2]:
import numpy as np
import os
# os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '1'
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

import cv2
import pudb
import datetime
import time
import sys
import fnmatch
import random
from imgaug import augmenters as iaa
import imgaug as ia
import matplotlib.pyplot as plt
from scipy import misc

sys.path.append('../3_svm_model/')
sys.path.append('../2_compare_detectors/')

from train import filenames_for_participants
from divide import divide_in

random.seed(0)
np.random.seed(0)

In [3]:
for kfold in range(5):
    print "loading images for" + str(kfold)
    # get all participants in training set to create a per participant ..
    # validation and test set.
    participants = []
    for root, dirnames, filenames in os.walk("../../images/train_set"):
        for filename in fnmatch.filter(filenames, '*.png'):
            part = filename.split(" - ")[0]
            if part not in participants:
                participants.append(part)
    participants.sort()

    # divide the participants in 5 parts (same as we used in the SVM k-fold 
    # validation), then loop through the participants and get the filenames
    # Subsequently the images are loaded and put into folds (k=5)
    participants_sliced = divide_in(participants, 5)
    folds = []
    for p in participants_sliced:
        filenames_pos, filenames_neg = \
            filenames_for_participants(p,
                os.walk("../../images/train_set"), cream=True)
        filenames_pos_mining, filenames_neg_mining = \
            filenames_for_participants(p,
                os.walk("../../images/classified_mining"), cream=True)
        filenames_pos = filenames_pos + filenames_pos_mining
        filenames_neg = filenames_neg + filenames_neg_mining
        filenames_pos.sort()
        filenames_neg.sort()

        images_pos = []
        images_neg = []

        for i, filename in enumerate(filenames_pos):
            img = cv2.imread(filename)
            img = cv2.resize(img, (64, 64))
            images_pos.append(img)
        for i, filename in enumerate(filenames_neg):
            img = cv2.imread(filename)
            img = cv2.resize(img, (64, 64))
            images_neg.append(img)

        folds.append([images_pos, images_neg])

    # Just like in the SVM training take the first fold as validation set (here 'test')
    train_set_pos = []
    train_set_neg = []
    test_set_pos = []
    test_set_neg = []
    for j, f_ in enumerate(folds):
        if j == kfold:
            test_set_pos = folds[j][0]
            test_set_neg = folds[j][1]
        else:
            train_set_pos += folds[j][0]
            train_set_neg += folds[j][1]

    # Create a array with labels and concatenate the positive and negative images
    # in one array (here wrongly called "hists"
    labels_train = np.zeros(len(train_set_pos)+len(train_set_neg))
    labels_train[len(train_set_pos):] = 1

    # One-hot the label array ([0, 1, 0] -> [[1, 0], [0, 1], [1, 0]]
    # For training this is done in the training loop
    labels_train = (labels_train[:,None] == np.arange(2)).astype(float)

    hists_train = np.concatenate((train_set_pos, train_set_neg))
    print "Size of the original train data: " + str(hists_train.shape)

    # Do the same for test set, and normalize the test set
    # Normalization of the training set is done after augmentation in the 
    # training loop.
    labels_test = np.zeros(len(test_set_pos)+len(test_set_neg))
    labels_test[len(test_set_pos):] = 1
    labels_test = (labels_test[:,None] == np.arange(2)).astype(float)

    hists_test = np.concatenate((test_set_pos, test_set_neg))
    hists_test = hists_test / 255.0 - 0.5

    np.save("hists_test_kfold_" + str(kfold), hists_test)
    np.save("labels_test_kfold_"  + str(kfold), labels_test)
    np.save("hists_train_kfold_"  + str(kfold), hists_train)
    np.save("labels_train_kfold_"  + str(kfold), labels_train)

loading images for0
Size of the original train data: (21691, 64, 64, 3)
loading images for1
Size of the original train data: (21270, 64, 64, 3)
loading images for2
Size of the original train data: (21500, 64, 64, 3)
loading images for3
Size of the original train data: (21940, 64, 64, 3)
loading images for4
Size of the original train data: (21139, 64, 64, 3)


In [1]:
print range(5)

[0, 1, 2, 3, 4]
